In [ ]:
from google.colab import drive 
drive.mount('/content/drive')
%cd /content/drive/My\ Drive/20.440/440_Final

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/content/drive/.shortcut-targets-by-id/1xhDshDvMt3Z8-IyFeZVRFUyn2VUSX-j4/440_Final


In [ ]:
# imports
import seaborn as sns
import pandas as pd
import numpy as np
!pip install scanpy
!pip install umap-learn
!pip install numba
!pip install mygene
import scanpy as sc
import scanpy.external as sce
import anndata as ad
import scipy as sp
import matplotlib.pyplot as plt
from matplotlib.pyplot import rc_context
import pickle
from natsort import index_natsorted
import mygene
mg = mygene.MyGeneInfo()
import re
from natsort import index_natsorted
!pip install mygene
#!pip uninstall statsmodels
#!pip install statsmodels
!pip install patsy
import statsmodels as sm
import statsmodels.stats.multitest
from patsy import dmatrices

In [ ]:
# read in pickle files of processed data (AnnData object)
def load_data(filename):
  infile = open(filename,'rb')
  data = pickle.load(infile)
  infile.close()
  return data

# convert to pandas dataframe
def ad_to_df(data_ad):
  df = data_ad.to_df()
  df.insert(0,'timepoint',data_ad.obs['timepoint'])
  df.insert(0,'dataset',data_ad.obs['dataset'])
  return df

Load preprocessed data from pickle file

In [ ]:
# load preprocessed data from pickle file, as both AnnData and DataFrame objects
all_ad = load_data('./Data/all_data_2000')
display(all_ad)
all_df = ad_to_df(all_ad)
display(all_df)

AnnData object with n_obs × n_vars = 79517 × 1999
    obs: 'dataset', 'timepoint', 'sample id', 'n_counts_all'
    var: 'n_counts', 'mean', 'std'
    uns: 'log1p'

,dataset,timepoint,1110038B12Rik,1500002F19Rik,1500015O10Rik,1600012P17Rik,1600015I10Rik,1600025M17Rik,1700003F12Rik,1700007K13Rik,1700008J07Rik,1700011M02Rik,1700013H16Rik,1700016D06Rik,1700064H15Rik,1700084F23Rik,1700086L19Rik,1810009A15Rik,1810059H22Rik,2210408F21Rik,2310007B03Rik,2310040G24Rik,2410006H16Rik,2510016D11Rik,2610307P16Rik,2610509F24Rik,2810002D19Rik,2810025M15Rik,3110035E14Rik,3830417A13Rik,4632415L05Rik,4921531C22Rik,4930412C18Rik,4930417H01Rik,4930426D05Rik,4930448N21Rik,4930505N22Rik,4930558K02Rik,4930593A02Rik,4933423P22Rik,...,Wdfy4,Wdr27,Wfdc1,Wfdc17,Wfdc18,Wfdc2,Wfdc21,Wfdc3,Wisp1,Wisp2,Wnt1,Wnt6,Xaf1,Xist,Ybx1,Yjefn3,Yy2,Zbp1,Zbtb21,Zcchc12,Zfas1,Zfp287,Zfp646,Zfp791,Zfp976,Zfp979,Zscan4d,Zyx,mt-Atp6,mt-Atp8,mt-Co1,mt-Co2,mt-Co3,mt-Cytb,mt-Nd1,mt-Nd3,mt-Nd4,mt-Nd4l,mt-Rnr1,mt-Rnr2
AAACGGGAGGCTATCT-1_GSM3964244_MEFs__0,babos,D0,-0.104723,-0.058508,-0.38649,-0.027101,-0.017621,-0.114295,-0.097796,-0.472211,-0.202382,-0.09235,-0.237124,-0.141272,-0.014234,-0.122963,-0.087723,-0.168790,-0.030397,-0.058139,-0.071311,-0.12699,-0.127693,-0.012890,-0.037164,-0.018748,-0.019838,-0.131498,-0.022385,-0.169303,1.434773,-0.038234,-0.042135,-0.021424,-0.015733,-0.025004,-0.044105,-0.084874,-0.013858,-0.017608,...,-0.075079,-0.034501,-0.036114,-0.033109,-0.023545,-0.593139,-0.026993,-0.066392,-0.044764,-0.314241,-0.042,-0.183442,-0.309666,-1.219304,-1.306783,-0.242934,-0.013617,-0.194919,-0.210152,-0.157674,-0.123821,-0.1076,1.777228,-0.054812,-0.030451,-0.025436,-0.014271,0.230400,2.667724,3.249765,2.164509,2.814948,2.697827,0.104211,0.082542,1.980935,-0.145684,2.647468,-0.117870,-0.150707
AAACGGGTCAATCTCT-1_GSM3964244_MEFs__0,babos,D0,-0.104723,-0.058508,-0.38649,-0.027101,-0.017621,-0.114295,-0.097796,-0.472211,1.439131,-0.09235,-0.237124,-0.141272,-0.014234,-0.122963,-0.087723,-0.168790,-0.030397,-0.058139,-0.071311,-0.12699,-0.127693,-0.012890,-0.037164,-0.018748,-0.019838,-0.131498,-0.022385,-0.169303,-0.258155,-0.038234,-0.042135,-0.021424,-0.015733,-0.025004,-0.044105,-0.084874,-0.013858,-0.017608,...,-0.075079,-0.034501,-0.036114,-0.033109,-0.023545,-0.593139,-0.026993,-0.066392,1.586428,-0.314241,-0.042,-0.183442,-0.309666,-0.256589,-1.665926,-0.242934,-0.013617,-0.194919,1.396220,-0.157674,-0.123821,-0.1076,-0.227782,-0.054812,-0.030451,-0.025436,-0.014271,1.511761,3.049047,6.290161,2.623946,3.200402,3.067121,0.622127,0.823205,3.635458,0.756943,4.093084,-0.117870,-0.150707
AAACGGGTCGTGGACC-1_GSM3964244_MEFs__0,babos,D0,-0.104723,-0.058508,-0.38649,-0.027101,-0.017621,-0.114295,-0.097796,-0.472211,-0.202382,-0.09235,-0.237124,-0.141272,-0.014234,-0.122963,-0.087723,-0.168790,-0.030397,-0.058139,-0.071311,-0.12699,-0.127693,-0.012890,-0.037164,-0.018748,-0.019838,-0.131498,-0.022385,-0.169303,-0.258155,-0.038234,-0.042135,-0.021424,-0.015733,-0.025004,-0.044105,-0.084874,-0.013858,-0.017608,...,-0.075079,-0.034501,-0.036114,-0.033109,-0.023545,-0.593139,-0.026993,-0.066392,0.192468,-0.314241,-0.042,-0.183442,-0.309666,-1.219304,-1.923130,-0.242934,-0.013617,-0.194919,-0.210152,-0.157674,-0.123821,-0.1076,-0.227782,-0.054812,-0.030451,-0.025436,-0.014271,0.115110,2.773601,2.865701,2.449381,2.791832,2.885589,0.013290,-0.010329,2.198749,0.114729,1.884841,-0.117870,-0.150707
AAACGGGTCTGAGTGT-1_GSM3964244_MEFs__0,babos,D0,-0.104723,-0.058508,-0.38649,-0.027101,-0.017621,-0.114295,-0.097796,-0.472211,-0.202382,-0.09235,-0.237124,-0.141272,-0.014234,-0.122963,-0.087723,-0.168790,-0.030397,-0.058139,-0.071311,-0.12699,-0.127693,-0.012890,-0.037164,-0.018748,-0.019838,-0.131498,-0.022385,-0.169303,-0.258155,-0.038234,-0.042135,-0.021424,-0.015733,-0.025004,-0.044105,-0.084874,-0.013858,-0.017608,...,-0.075079,-0.034501,-0.036114,-0.033109,-0.023545,-0.593139,-0.026993,-0.066392,0.233293,-0.314241,-0.042,-0.183442,0.852771,0.706413,-1.755061,-0.242934,-0.013617,-0.194919,-0.210152,-0.157674,-0.123821,-0.1076,-0.227782,-0.054812,-0.030451,-0.025436,-0.014271,-0.689880,2.393834,3.022477,2.112087,2.464108,2.378313,-0.220

In [ ]:
# compute mean expression for genes by timepoint and dataset

def mean_by_time(df,name):
  means = df.loc[all_df['dataset']==name].groupby(['timepoint']).mean()
  means.sort_values(by='timepoint', key=lambda x:np.argsort(index_natsorted(x)),
                    inplace=True)
  return means

babos_means = mean_by_time(all_df,'babos')
shie_means = mean_by_time(all_df,'shiebinger')
shie_means_tr = shie_means.loc[(shie_means.index=='D0') | 
                               (shie_means.index=='D4') | 
                               (shie_means.index=='D8') | 
                               (shie_means.index=='D16')]
fran_means = mean_by_time(all_df,'fran')
fran_means_tr = fran_means.loc[(fran_means.index=='0h') | 
                               (fran_means.index=='D2') | 
                               (fran_means.index=='D4') | 
                               (fran_means.index=='D8')]

Correlation coefficient calculations with Benjamini-Hochberg multiple hypothesis correction

In [ ]:
# perform Spearman correlation analysis across genes for shared timepoints 
#  between pairs of datasets

times = {'babos': ['D0','D4','D8','D14'],
         'shie': ['D0','D4','D8','D16'],
         'fran': ['0h','D2','D4','D8']}

spearman = pd.DataFrame(columns=['dataset1','dataset2','timepoint',
                                 'Spearman','p-value'])

for i in range(len(times['babos'])):

  # get gene means at the given timepoint for each dataset
  babos_mt = babos_means.loc[(babos_means.index==times['babos'][i])]
  shie_mt = shie_means.loc[(shie_means.index==times['shie'][i])]
  fran_mt = fran_means.loc[(fran_means.index==times['fran'][i])]

  # compute Spearman correlation coefficient 
  spearman_bs = sp.stats.spearmanr(np.array(babos_mt)[0],np.array(shie_mt)[0])
  spearman.loc[len(spearman.index)] = ['babos','shie',
                                       (times['babos'][i],times['shie'][i]),
                                       spearman_bs[0],spearman_bs[1]]
  
  spearman_bf = sp.stats.spearmanr(np.array(babos_mt)[0],np.array(fran_mt)[0])
  spearman.loc[len(spearman.index)] = ['babos','fran',
                                       (times['babos'][i],times['fran'][i]),
                                       spearman_bf[0],spearman_bf[1]]
  
  spearman_sf = sp.stats.spearmanr(np.array(shie_mt)[0],np.array(fran_mt)[0])
  spearman.loc[len(spearman.index)] = ['shie','fran',
                                       (times['shie'][i],times['fran'][i]),
                                       spearman_sf[0],spearman_sf[1]]
  
benjamini_hochberg = sm.stats.multitest.multipletests(spearman['p-value'], 
                                                      alpha=0.05, 
                                                      method='fdr_bh')
spearman.insert(len(spearman.columns),'corrected p-value',benjamini_hochberg[1])
spearman.insert(len(spearman.columns),'significant',benjamini_hochberg[0])
display(spearman.sort_values(by=['dataset1','dataset2']))

,dataset1,dataset2,timepoint,Spearman,p-value,corrected p-value,significant
1,babos,fran,"(D0, 0h)",0.048195,3.118320e-02,4.677480e-02,True
4,babos,fran,"(D4, D2)",0.018464,4.093265e-01,4.465380e-01,False
7,babos,fran,"(D8, D4)",-0.059050,8.271614e-03,1.654323e-02,True
10,babos,fran,"(D14, D8)",-0.141985,1.811667e-10,7.246666e-10,True
0,babos,shie,"(D0, D0)",0.521241,1.136262e-139,1.363514e-138,True
3,babos,shie,"(D4, D4)",0.182884,1.703789e-16,1.022273e-15,True
6,babos,shie,"(D8, D8)",-0.069035,2.012912e-03,4.830990e-03,True
9,babos,shie,"(D14, D16)",0.011864,5.960306e-01,5.960306e-01,False
2,shie,fran,"(D0, 0h)",-0.020688,3.552292e-01,4.262751e-01,False
5,shie,fran,"(D4, D2)",0.034176,1.266401e-01,1.688535e-01,False


In [ ]:
grouped = all_df.groupby(by='dataset')
babos_df = grouped.get_group('babos')
shie_df = grouped.get_group('shiebinger')
fran_df = grouped.get_group('fran')

In [ ]:
# perform Pearson correlation analysis for genes across timepoints

def pearson_for_genes(means1,means2):
  
  pearson = []
  for g in means1.columns:

    pear = sp.stats.pearsonr(means1[g],means2[g])
    pearson.append(pear)
  
  # display top 25 correlated genes
  pearson_df = pd.DataFrame(pearson, index=means1.columns, 
                            columns=['Pearson','p-value'])
  
  # correct for multiple hypotheses
  pearson_df.dropna(axis=0,inplace=True)
  benjamini_hochberg = sm.stats.multitest.multipletests(pearson_df['p-value'], 
                                                      alpha=0.05, 
                                                      method='fdr_bh')
  pearson_df.insert(len(pearson_df.columns),'corrected p-value',
                    benjamini_hochberg[1])
  pearson_df.insert(len(pearson_df.columns),'significant',benjamini_hochberg[0])

  display(pearson_df.sort_values(by='Pearson',ascending=False)[:10])
  
  return pearson_df

pearson_bs = pearson_for_genes(babos_means,shie_means_tr)
pearson_bf = pearson_for_genes(babos_means,fran_means_tr)
pearson_sf = pearson_for_genes(shie_means_tr,fran_means_tr)

/usr/local/lib/python3.7/dist-packages/scipy/stats/stats.py:3508: PearsonRConstantInputWarning: An input array is constant; the correlation coefficent is not defined.
  warnings.warn(PearsonRConstantInputWarning())


,Pearson,p-value,corrected p-value,significant
Ptgds,0.999914,0.000086,0.047897,True
Tcf15,0.999908,0.000092,0.047897,True
C1qtnf3,0.999877,0.000123,0.047897,True
Snhg11,0.999676,0.000324,0.094895,False
Th,0.999547,0.000453,0.101786,False
Elf3,0.999402,0.000598,0.101786,False
F3,0.999367,0.000633,0.101786,False
Nkx6-2,0.999305,0.000695,0.101786,False
Bex1,0.999203,0.000797,0.103739,False
Lncpint,0.998620,0.001380,0.161575,False


/usr/local/lib/python3.7/dist-packages/scipy/stats/stats.py:3508: PearsonRConstantInputWarning: An input array is constant; the correlation coefficent is not defined.
  warnings.warn(PearsonRConstantInputWarning())


,Pearson,p-value,corrected p-value,significant
Gpx2,1.000000,0.000000,0.000000,True
Elf3,0.999781,0.000219,0.102533,False
Akr1c18,0.999124,0.000876,0.273501,False
Snhg11,0.998589,0.001411,0.320675,False
Atp2b1,0.998166,0.001834,0.320675,False
Igfbp4,0.997404,0.002596,0.320675,False
Ckmt1,0.997006,0.002994,0.320675,False
Atp8b4,0.996307,0.003693,0.320675,False
Slc2a3,0.996306,0.003694,0.320675,False
Ndrg1,0.995951,0.004049,0.320675,False


/usr/local/lib/python3.7/dist-packages/scipy/stats/stats.py:3508: PearsonRConstantInputWarning: An input array is constant; the correlation coefficent is not defined.
  warnings.warn(PearsonRConstantInputWarning())


,Pearson,p-value,corrected p-value,significant
Ptprtos,1.000000,1.110223e-16,1.117995e-13,True
Dppa3,1.000000,2.800433e-07,1.064363e-04,True
Rhox9,1.000000,3.170891e-07,1.064363e-04,True
Rhox5,0.999999,6.574426e-07,1.549688e-04,True
Dppa5a,0.999999,7.694579e-07,1.549688e-04,True
Trap1a,0.999998,1.569601e-06,2.634313e-04,True
Sct,0.999998,2.380088e-06,3.423927e-04,True
Phlda2,0.999987,1.255318e-05,1.441200e-03,True
Khdc3,0.999987,1.288064e-05,1.441200e-03,True
Tdgf1,0.999977,2.294570e-05,2.310632e-03,True


In [ ]:
# Analyze GSEA results
def get_sig(df,alpha=0.01,fdr=0.25):
  return set(df.loc[(df['FDR q-val']<fdr) & (df['NOM p-val'] < alpha),'NAME'])

gsea_bs_go_pos = pd.read_csv('./GSEA/means_bs_pearson_peak_GO.Gsea.1620534558050/gsea_report_for_pairs_means_peak_pos_1620534558050.tsv',sep='\t')
gsea_bs_go_pos_sig = get_sig(gsea_bs_go_pos)
gsea_bs_go_neg = pd.read_csv('./GSEA/means_bs_pearson_peak_GO.Gsea.1620534558050/gsea_report_for_pairs_means_peak_neg_1620534558050.tsv',sep='\t')
gsea_bs_go_neg_sig = get_sig(gsea_bs_go_neg)

gsea_sf_go_pos = pd.read_csv('./GSEA/means_sf_pearson_peak_GO.Gsea.1620534854135/gsea_report_for_pairs_means_peak_pos_1620534854135.tsv',sep='\t')
gsea_sf_go_pos_sig = get_sig(gsea_sf_go_pos)
gsea_sf_go_neg = pd.read_csv('./GSEA/means_sf_pearson_peak_GO.Gsea.1620534854135/gsea_report_for_pairs_means_peak_neg_1620534854135.tsv',sep='\t')
gsea_sf_go_neg_sig = get_sig(gsea_sf_go_neg)

gsea_bf_go_pos = pd.read_csv('./GSEA/means_bf_pearson_peak_GO.Gsea.1620534842054/gsea_report_for_pairs_means_peak_pos_1620534842054.tsv',sep='\t')
gsea_bf_go_pos_sig = get_sig(gsea_bf_go_pos)
gsea_bf_go_neg = pd.read_csv('./GSEA/means_bf_pearson_peak_GO.Gsea.1620534842054/gsea_report_for_pairs_means_peak_neg_1620534842054.tsv',sep='\t')
gsea_bf_go_neg_sig = get_sig(gsea_bf_go_neg)

In [ ]:
display(gsea_bs_go_neg_sig)
display(gsea_bf_go_neg_sig)
display(gsea_sf_go_neg_sig)

In [ ]:
sig_go_pos_all = gsea_bs_go_pos_sig.intersection(gsea_bf_go_pos_sig,gsea_sf_go_pos_sig)
display(sig_go_pos_all)

sig_go_neg_all = gsea_bs_go_neg_sig.intersection(gsea_bf_go_neg_sig,gsea_sf_go_neg_sig)
display(sig_go_neg_all)

{'GOBP_COTRANSLATIONAL_PROTEIN_TARGETING_TO_MEMBRANE',
 'GOBP_CYTOPLASMIC_TRANSLATION',
 'GOBP_ESTABLISHMENT_OF_PROTEIN_LOCALIZATION_TO_ENDOPLASMIC_RETICULUM',
 'GOBP_NUCLEAR_TRANSCRIBED_MRNA_CATABOLIC_PROCESS',
 'GOBP_NUCLEAR_TRANSCRIBED_MRNA_CATABOLIC_PROCESS_NONSENSE_MEDIATED_DECAY',
 'GOBP_PROTEIN_LOCALIZATION_TO_ENDOPLASMIC_RETICULUM',
 'GOBP_PROTEIN_TARGETING_TO_MEMBRANE',
 'GOBP_RESPONSE_TO_XENOBIOTIC_STIMULUS',
 'GOBP_RNA_CATABOLIC_PROCESS',
 'GOBP_TRANSLATIONAL_INITIATION',
 'GOBP_VIRAL_GENE_EXPRESSION',
 'GOCC_CYTOSOLIC_RIBOSOME',
 'GOCC_LARGE_RIBOSOMAL_SUBUNIT',
 'GOCC_MITOCHONDRIAL_PROTEIN_CONTAINING_COMPLEX',
 'GOCC_RIBOSOMAL_SUBUNIT',
 'GOMF_RNA_BINDING',
 'GOMF_STRUCTURAL_CONSTITUENT_OF_RIBOSOME'}

set()

In [ ]:
gsea_bs_motif_pos = pd.read_csv('./GSEA/means_bs_pearson_peak_motif.Gsea.1620534902359/gsea_report_for_pairs_means_peak_pos_1620534902359.tsv',sep='\t')
gsea_bs_motif_pos_sig = get_sig(gsea_bs_motif_pos,alpha=0.05)
gsea_bs_motif_neg = pd.read_csv('./GSEA/means_bs_pearson_peak_motif.Gsea.1620534902359/gsea_report_for_pairs_means_peak_neg_1620534902359.tsv',sep='\t')
gsea_bs_motif_neg_sig = get_sig(gsea_bs_motif_neg,alpha=0.05)

gsea_sf_motif_pos = pd.read_csv('./GSEA/means_sf_pearson_peak_motif.Gsea.1620534917714/gsea_report_for_pairs_means_peak_pos_1620534917714.tsv',sep='\t')
gsea_sf_motif_pos_sig = get_sig(gsea_sf_motif_pos,alpha=0.05)
gsea_sf_motif_neg = pd.read_csv('./GSEA/means_sf_pearson_peak_motif.Gsea.1620534917714/gsea_report_for_pairs_means_peak_neg_1620534917714.tsv',sep='\t')
gsea_sf_motif_neg_sig = get_sig(gsea_sf_motif_neg,alpha=0.05)

gsea_bf_motif_pos = pd.read_csv('./GSEA/means_bf_pearson_peak_motif.Gsea.1620534927232/gsea_report_for_pairs_means_peak_pos_1620534927232.tsv',sep='\t')
gsea_bf_motif_pos_sig = get_sig(gsea_bf_motif_pos,alpha=0.05)
gsea_bf_motif_neg = pd.read_csv('./GSEA/means_bf_pearson_peak_motif.Gsea.1620534927232/gsea_report_for_pairs_means_peak_neg_1620534927232.tsv',sep='\t')
gsea_bf_motif_neg_sig = get_sig(gsea_bf_motif_neg,alpha=0.05)

In [ ]:
sig_motif_pos_all = gsea_bs_motif_pos_sig.intersection(gsea_bf_motif_pos_sig,gsea_sf_motif_pos_sig)
display(sig_motif_pos_all)

sig_motif_neg_all = gsea_bs_motif_neg_sig.intersection(gsea_bf_motif_neg_sig,gsea_sf_motif_neg_sig)
display(sig_motif_neg_all)

{'E2F_Q6_01', 'MIR6736_3P', 'RUVBL1_TARGET_GENES', 'ZNF260_TARGET_GENES'}

set()

In [ ]:
def get_intersect(set_list):
  intersect = np.zeros((len(set_list),len(set_list)))

  for i,set1 in enumerate(set_list):
    for j,set2 in enumerate(set_list):
      intersect[i,j] = len(set1.intersection(set2))

  return intersect

sig_go_pos = [gsea_bs_go_pos_sig, gsea_bf_go_pos_sig, gsea_sf_go_pos_sig]
sig_go_neg = [gsea_bs_go_neg_sig, gsea_bf_go_neg_sig, gsea_sf_go_neg_sig]

intersect_go_pos = get_intersect(sig_go_pos)
print(intersect_go_pos)
intersect_go_neg = get_intersect(sig_go_neg)
print(intersect_go_neg)

[[74. 17. 47.]
 [17. 27. 19.]
 [47. 19. 74.]]
[[81.  0. 18.]
 [ 0.  8.  0.]
 [18.  0. 26.]]


In [ ]:
sig_motif_pos = [gsea_bs_motif_pos_sig, gsea_bf_motif_pos_sig, gsea_sf_motif_pos_sig]
sig_motif_neg = [gsea_bs_motif_neg_sig, gsea_bf_motif_neg_sig, gsea_sf_motif_neg_sig]

intersect_motif_pos = get_intersect(sig_motif_pos)
print(intersect_motif_pos)
intersect_motif_neg = get_intersect(sig_motif_neg)
print(intersect_motif_neg)

[[121.   4.  13.]
 [  4.   6.   6.]
 [ 13.   6.  18.]]
[[0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 1.]]


In [ ]:
# GSEA all GO

gsea_all_go_pos = pd.read_csv('./GSEA/means_all_pearson_peak_GO.Gsea.1620771247543/gsea_report_for_triple_means_peak_pos_1620771247543.tsv',sep='\t')
gsea_all_go_pos_sig = get_sig(gsea_all_go_pos)
gsea_all_go_neg = pd.read_csv('./GSEA/means_all_pearson_peak_GO.Gsea.1620771247543/gsea_report_for_triple_means_peak_neg_1620771247543.tsv',sep='\t')
gsea_all_go_neg_sig = get_sig(gsea_all_go_neg)

In [ ]:
display(gsea_all_go_pos_sig.intersection(sig_go_pos_all))
print(len(gsea_all_go_pos_sig))
print(len(gsea_all_go_neg_sig))

{'GOBP_COTRANSLATIONAL_PROTEIN_TARGETING_TO_MEMBRANE',
 'GOBP_CYTOPLASMIC_TRANSLATION',
 'GOBP_ESTABLISHMENT_OF_PROTEIN_LOCALIZATION_TO_ENDOPLASMIC_RETICULUM',
 'GOBP_NUCLEAR_TRANSCRIBED_MRNA_CATABOLIC_PROCESS',
 'GOBP_NUCLEAR_TRANSCRIBED_MRNA_CATABOLIC_PROCESS_NONSENSE_MEDIATED_DECAY',
 'GOBP_PROTEIN_LOCALIZATION_TO_ENDOPLASMIC_RETICULUM',
 'GOBP_PROTEIN_TARGETING_TO_MEMBRANE',
 'GOBP_RESPONSE_TO_XENOBIOTIC_STIMULUS',
 'GOBP_RNA_CATABOLIC_PROCESS',
 'GOBP_TRANSLATIONAL_INITIATION',
 'GOBP_VIRAL_GENE_EXPRESSION',
 'GOCC_CYTOSOLIC_RIBOSOME',
 'GOCC_LARGE_RIBOSOMAL_SUBUNIT',
 'GOCC_MITOCHONDRIAL_PROTEIN_CONTAINING_COMPLEX',
 'GOCC_RIBOSOMAL_SUBUNIT',
 'GOMF_RNA_BINDING',
 'GOMF_STRUCTURAL_CONSTITUENT_OF_RIBOSOME'}

49
59


In [ ]:
ranks_pos = [gsea_all_go_pos.loc[gsea_all_go_pos['NAME'] == gs].index for gs in sig_go_pos_all]
display(ranks_pos)

[Int64Index([15], dtype='int64'),
 Int64Index([3], dtype='int64'),
 Int64Index([13], dtype='int64'),
 Int64Index([5], dtype='int64'),
 Int64Index([10], dtype='int64'),
 Int64Index([32], dtype='int64'),
 Int64Index([1], dtype='int64'),
 Int64Index([8], dtype='int64'),
 Int64Index([4], dtype='int64'),
 Int64Index([14], dtype='int64'),
 Int64Index([6], dtype='int64'),
 Int64Index([0], dtype='int64'),
 Int64Index([11], dtype='int64'),
 Int64Index([7], dtype='int64'),
 Int64Index([28], dtype='int64'),
 Int64Index([2], dtype='int64'),
 Int64Index([9], dtype='int64')]

In [ ]:
# GSEA all motif

gsea_all_motif_pos = pd.read_csv('./GSEA/means_all_pearson_peak_motif.Gsea.1620774941636/gsea_report_for_triple_means_peak_pos_1620774941636.tsv',sep='\t')
gsea_all_motif_pos_sig = get_sig(gsea_all_motif_pos)
gsea_all_motif_neg = pd.read_csv('./GSEA/means_all_pearson_peak_motif.Gsea.1620774941636/gsea_report_for_triple_means_peak_neg_1620774941636.tsv',sep='\t')
gsea_all_motif_neg_sig = get_sig(gsea_all_motif_neg,alpha=0.05)

In [ ]:
display(gsea_all_motif_pos_sig)

display(gsea_all_motif_pos_sig.intersection(sig_motif_pos_all))
print(len(gsea_all_motif_pos_sig))
print(len(gsea_all_motif_neg_sig))

ranks_pos_m = [gsea_all_motif_pos.loc[gsea_all_motif_pos['NAME'] == gs].index for gs in sig_motif_pos_all]
display(ranks_pos_m)

{'ARNT2_TARGET_GENES',
 'E2F_Q4_01',
 'E2F_Q6_01',
 'FOXR2_TARGET_GENES',
 'MIR4786_3P',
 'MIR6736_3P',
 'MIR6839_3P',
 'RUVBL1_TARGET_GENES',
 'ZNF260_TARGET_GENES',
 'ZNF711_TARGET_GENES',
 'ZZZ3_TARGET_GENES'}

{'E2F_Q6_01', 'MIR6736_3P', 'RUVBL1_TARGET_GENES', 'ZNF260_TARGET_GENES'}

11
0


[Int64Index([3], dtype='int64'),
 Int64Index([1], dtype='int64'),
 Int64Index([10], dtype='int64'),
 Int64Index([5], dtype='int64')]

In [ ]:
# GSEA all reactome

gsea_all_reactome_pos = pd.read_csv('./GSEA/means_all_pearson_peak_reactome.Gsea.1620776243562/gsea_report_for_triple_means_peak_pos_1620776243562.tsv',sep='\t')
gsea_all_reactome_pos_sig = get_sig(gsea_all_reactome_pos)
gsea_all_reactome_neg = pd.read_csv('./GSEA/means_all_pearson_peak_reactome.Gsea.1620776243562/gsea_report_for_triple_means_peak_neg_1620776243562.tsv',sep='\t')
gsea_all_reactome_neg_sig = get_sig(gsea_all_reactome_neg)

print(len(gsea_all_reactome_pos_sig))
print(len(gsea_all_reactome_neg_sig))

23
8
